In [1]:
import pandas as pd
import numpy as np
import GEOparse, re, copy
import csv

In [3]:
data_f = pd.read_csv("/data/users-workspace/ruchika.sharma/Alzheimers/Microarray/GSE_ids.csv", header = None)
GSEID = data_f.values.tolist()
flat_list = []
for sublist in GSEID:
    for item in sublist:
        flat_list.append(item)
unique_GSE = set(flat_list)

In [4]:
unique_GSE

{'GSE110226',
 'GSE110298',
 'GSE117584',
 'GSE117585',
 'GSE117586',
 'GSE122063',
 'GSE16759',
 'GSE18920',
 'GSE36980',
 'GSE37263',
 'GSE37264',
 'GSE48350',
 'GSE53890',
 'GSE58349',
 'GSE58364',
 'GSE84422',
 'GSE93885'}

In [5]:
def getGEO(unique_GSE):
    try:
        gse = GEOparse.get_GEO(geo = unique_GSE, destdir = "/data/users-workspace/ruchika.sharma/Alzheimers/Microarray", silent=True)
    except Exception as e:
        print(gse)
    try:
        gpl_name = ([key for key in gse.gpls.keys()])[0]
        gpl = GEOparse.get_GEO(geo = gpl_name, destdir = "/data/users-workspace/ruchika.sharma/Alzheimers/Microarray", silent=True, annotate_gpl = True)
    except Exception as e:
        print(gpl)
    return gse, gpl

In [6]:
def prep_deg_input(gse, gse_id):
    #For expression data
    gpl_name = ([key for key in gse.gpls.keys()])[0]
    pivoted_samples = gse.pivot_samples('VALUE')
    samples = gse.pivot_samples("VALUE")
    gse.pivot_samples('VALUE')
    gse.gpls[gpl_name].table.head(50)
    gse_annotated = pivoted_samples.reset_index().merge(gse.gpls[gpl_name].table[["ID","Gene Symbol"]], left_on='ID_REF', right_on="ID").set_index('ID_REF')
    # gse_annotated = pivoted_samples.reset_index().merge(df[["ID", "Gene Symbol"]], left_on='ID_REF', right_on="ID").set_index('ID_REF')
    gse_annotated.reset_index(level=0, inplace=True)
    gse_annotated.dropna()
    del gse_annotated["ID_REF"]
    del gse_annotated["ID"]
    gse_annotated = gse_annotated.dropna(subset=["Gene Symbol"])#Replace
    gse_annotated = gse_annotated.rename(columns={"Gene Symbol": "Gene"})
    gse_annotated = gse_annotated[~gse_annotated.Gene.str.contains("///")]
    #gse_annotated['Gene'] = gse_annotated['Gene'].str.split('//').str[1]
    gse_annotated = gse_annotated.groupby("Gene").median()
    #gse_annotated = gse_annotated.apply(np.log2, axis=0)
    gse_annotated.reset_index(level=0, inplace=True)
    #gse_annotated['Gene'] = gse_annotated['Gene'].str.split('//').str[1]
    gse_annotated = gse_annotated.iloc[1:]
    # #gse_annotated = gse_annotated[gse_annotated['Gene'] != '---']
    # #gse_annotated = gse_annotated.iloc[1:]
    gse_annotated.reset_index(drop=True, inplace=True)
    
    #gse_annotated
    gse_annotated.to_csv(f'/data/users-workspace/ruchika.sharma/Alzheimers/Microarray/{gse_id}_FPKM_ExpData.csv', index=False)

In [7]:
error_gse = []
for i in unique_GSE:
    gse = getGEO(i)
    try:
        prep_deg_input(gse, i)
    except Exception:
        print(i)
    error_gse.append(i)

UnboundLocalError: local variable 'gpl' referenced before assignment

In [32]:
####Sample_info
df2 = pd.read_csv("sample_info.csv", sep = '\t')
df3 = df2.dropna()
for i in unique_GSE:
    #print(i)
    study_specific= df3[df3['ExperimentID'].isin({i})]
    study_specific.to_csv(f'/data/users-workspace/ruchika.sharma/discoveric/microarray/analysis/{i}_Sample_info.csv', index = False)

In [31]:
df = pd.read_csv("gse_with_error", header = None)

In [18]:
# with open('gse_with_error', 'w', newline='') as myfile:
#      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      wr.writerow(error_gse)

In [19]:
gse_error = df.values.tolist()

In [20]:
len(gse_error)

17

NameError: name 'pd' is not defined

In [23]:
def prep_deg_input(gse, gse_id):
    #For expression data
    gpl_name = ([key for key in gse.gpls.keys()])[0]
    pivoted_samples = gse.pivot_samples('VALUE')
    samples = gse.pivot_samples("VALUE")
    gse.pivot_samples('VALUE')
    gse.gpls[gpl_name].table.head(50)
    gse_annotated = pivoted_samples.reset_index().merge(gse.gpls[gpl_name].table[["ID","Gene name"]], left_on='ID_REF', right_on="ID").set_index('ID_REF')
    # gse_annotated = pivoted_samples.reset_index().merge(df[["ID", "Gene Symbol"]], left_on='ID_REF', right_on="ID").set_index('ID_REF')
    gse_annotated.reset_index(level=0, inplace=True)
    gse_annotated.dropna()
    del gse_annotated["ID_REF"]
    del gse_annotated["ID"]
    gse_annotated = gse_annotated.dropna(subset=["Gene name"])#Replace
    gse_annotated = gse_annotated.rename(columns={"Gene name": "Gene"})
    gse_annotated = gse_annotated[~gse_annotated.Gene.str.contains("///")]
    #gse_annotated['Gene'] = gse_annotated['Gene'].str.split('//').str[1]
    gse_annotated = gse_annotated.groupby("Gene").median()
    #gse_annotated = gse_annotated.apply(np.log2, axis=0)
    gse_annotated.reset_index(level=0, inplace=True)
    #gse_annotated['Gene'] = gse_annotated['Gene'].str.split('//').str[1]
    gse_annotated = gse_annotated.iloc[1:]
    # #gse_annotated = gse_annotated[gse_annotated['Gene'] != '---']
    # #gse_annotated = gse_annotated.iloc[1:]
    gse_annotated.reset_index(drop=True, inplace=True)
    
    #gse_annotated
    gse_annotated.to_csv(f'/data/users-workspace/ruchika.sharma/discoveric/microarray/analysis/{gse_id}_FPKM_ExpData.csv', index=False)

In [24]:
error_gse = []
for i in gse_error:
    try:
        prep_deg_input(gse, i)
    except Exception:
        print(i)
    error_gse.append(i)

['GSE75988']
['GSE14069']
['GSE86341']
['GSE49474']
['GSE17389']
['GSE15783']
['GSE11509']
['GSE59408']
['GSE57084']
['GSE25478']
['GSE6712']
['GSE47221']
['GSE30698']
['GSE22060']
['GSE110628']
['GSE87627']
['GSE28152']
